# Week 3 Assignment

In [7]:
# !pip install beautifulsoup4 

## Code to parse the wikipage and build dataframe

In [8]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
filename = 'wiki_scrape.csv'

# csv_file = open(filename, 'w')
# csv_writer = csv.writer(csv_file)
# headers = ['PostCode', 'Borough', 'Neighbourhood']

# get the table and its tbody from html 
wikitable = soup.find('table', class_='wikitable')
wikitbody = wikitable.find('tbody')
# loop through all rows in tbody
dataList=[]
for trow in wikitbody.find_all('tr'):
    # the tbody is returning first column as th - handle it by using as first row
    cols = trow.find_all('th')
    if (cols != []) : 
        headers=[cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()]        
        # csv_writer.writerow(headers)
        
    else : 
        # now work on the columns 
        cols = trow.find_all('td')                 
        
        postcode = cols[0].text.strip()       
        
        # get borough value as it can be either a tag or just text
        if (cols[1].a != None) :  borough = cols[1].a.text.strip() 
        else :  borough = cols[1].text.strip()
       
        # get neighborhood and if it's value is 'not assigned', use borough value
        neighborhood = cols[2].text.strip() 
        if (neighborhood.lower() == 'not assigned') : neighborhood = borough
        
        # Skip loading 'not assigned' borough 
        if ( borough.lower() != "not assigned") :  
            # csv_writer.writerow([postcode, borough, neighborhood])
            dataList.append([postcode, borough, neighborhood])        
        
# csv_file.close()
# tor_df = pd.read_csv(filename)
tor_df = pd.DataFrame(dataList, columns=headers)
tor_df.head()
                      

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


###  combine neighbourhoods with a comma for the duplicate postcodes 

In [9]:
# combine neighbourhoods with a comma for the duplicate postcodes 
tor_df = tor_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
tor_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### print shape of dataframe

In [10]:
# shape of dataframe
print('The dataframe has {} boroughs and {} postcodes.'.format(
        len(tor_df['Borough'].unique()),
        tor_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 postcodes.
